In [2]:
import pandas as pd

from epilepsy_prediction import load_prediction_model, data_preprocess,fuse_string,evaluate_model,load_imputation_model
from epilepsy_prediction.imputation import column_imputer

from copy import deepcopy
from pandas import read_csv

# How to predict

Loading the model, we can select between: Carbamazepine, Gabapentin, Lamotrigine, Levetiracetam, Oxcarbazepine, Phenytoin,Topiramate,Valproate. 

In [3]:
def load_data(change_name_file,data_file_name,outcome_col="outcome"):
    data_df = pd.read_csv(data_file_name, index_col=0)
    new_name_df = pd.read_csv(change_name_file,index_col=0)
    data_df.rename(new_name_df['marketscan'].to_dict(),inplace=True,axis=1)
    y_train = data_df.loc[data_df["test_train"]=="train",outcome_col]
    X_train = data_df.loc[data_df["test_train"]=="train",~( data_df.columns.isin([outcome_col,"test_train"]))]
    y_test = data_df.loc[~(data_df["test_train"]=="train"),outcome_col]
    X_test = data_df.loc[~(data_df["test_train"]=="train"),~ (data_df.columns.isin([outcome_col,"test_train"]))]
    return X_train,y_train,X_test,y_test


In [4]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_epilepsy_19.01.23.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_any")

Check the correctness of input.

In [5]:
col_name_tbl = read_csv(r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv")
feature_names = col_name_tbl.iloc[:, 1]
assert not all(X_train.columns.isin(feature_names)), "Indeed some features can be missing"
assert all(feature_names.isin(X_train.columns)), "Some features is missing from training data"

In [7]:
drug_name="carbamazepine"
clf = load_prediction_model(drug_name)

In [13]:
X = data_preprocess(X_train,impute_drug=drug_name,replace_mean=True)
Xs = data_preprocess(X_test,impute_drug=drug_name,replace_mean=True)

In [15]:
X

,ADMISSION_EP_count,ADMISSION_EP_daysToInddate_min,ADMISSION_EP_indicator,ANY_ADVERSE_EVENT_EP_count,ANY_ADVERSE_EVENT_EP_daysToInddate_min,ANY_ADVERSE_EVENT_EP_indicator,ANY_EEG_count,ANY_EEG_daysToInddate_min,ANY_EEG_indicator,CARBAMAZEPINE_daysToInddate_min,...,stdprov_Therapy (Physical)_count,stdprov_Thoracic Surgery_count,stdprov_Transplant Surgery_count,stdprov_Transportation_count,stdprov_Traumatic Surgery_count,stdprov_Treatment Center_count,stdprov_Urgent Care Facility_count,stdprov_Urology_count,stdprov_Vision Center_count,treatment_arm
id,,,,,,,,,,,,,,,,,,,,,
EGVD004251_2007-12-18,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD008736_2019-05-31,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD010712_2005-04-13,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD011701_2005-01-24,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD015490_2019-09-16,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EGVD959888_2018-05-14,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,1,0,NaN,NaN
EGVD961237_2009-04-20,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD961237_2014-06-25,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN


And we can also use the model for training using it's fit method. 

In [18]:
def impute_columns_1(X, imputed_values,replace_mean=True):
    """
    Add X with missing columns, the values entered are the mean values
    Args:
        X (pd.DataFrame): The feature matrix
        imputed_values (pd.DataFrame): A the mean feature values of each drug data set
    Returns:
        pd.DataFrame: An imputed data frame
    """
    impute_cols = set(imputed_values.index) - set(X.columns)
    for col in impute_cols:
        if replace_mean:
            X[col] = imputed_values[col]
        else:
            X[col] = np.nan
    return X.loc[:, imputed_values.index]


In [25]:
def data_preprocess_1(X, impute_drug=None, impute_file_name=f'../csv/mean_pred_cals.csv',replace_mean=True):
    """
    load the data and impute it if required
    Args:
        data_file_name (str): The location of a pickle containing  a dictionary with two keys
        one X the data set and another y the outcomes
        impute_drug (str, optional): The drug to impute by Defaults to None.
        impute_file_name (str, optional): the location of the impute data. Defaults to f'../csv/mean_pred_cals.csv'.
    Returns:
        tuple: a tuple where the first element is the imputed data frame and the second is the outcome
    """
    impute_data = pd.read_csv(impute_file_name, index_col=0)
    display(impute_data)
    if not impute_drug is None:
        print(impute_drug)
        X = impute_columns_1(X, impute_data.loc[impute_drug.lower(), :],replace_mean=replace_mean)
    return X.loc[:, impute_data.columns]


In [27]:
read_csv('../csv/mean_pred_cals.csv')

,Unnamed: 0,ADMISSION_EP_count,ADMISSION_EP_daysToInddate_min,ADMISSION_EP_indicator,ANY_ADVERSE_EVENT_EP_count,ANY_ADVERSE_EVENT_EP_daysToInddate_min,ANY_ADVERSE_EVENT_EP_indicator,ANY_EEG_count,ANY_EEG_daysToInddate_min,ANY_EEG_indicator,...,stdprov_Therapy (Physical)_count,stdprov_Thoracic Surgery_count,stdprov_Transplant Surgery_count,stdprov_Transportation_count,stdprov_Traumatic Surgery_count,stdprov_Treatment Center_count,stdprov_Urgent Care Facility_count,stdprov_Urology_count,stdprov_Vision Center_count,treatment_arm
0,carbamazepine,0.070428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,valproate,0.104927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,oxcarbazepine,0.143192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,lamotrigine,0.106131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,levetiracetam,0.165251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,topiramate,0.068354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,gabapentin,0.040755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,phenytoin,0.166232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
drug_name

'carbamazepine'

In [26]:
data_preprocess_1(X_test,impute_drug=drug_name, impute_file_name='õige asukoht' replace_mean=True)

,ADMISSION_EP_count,ADMISSION_EP_daysToInddate_min,ADMISSION_EP_indicator,ANY_ADVERSE_EVENT_EP_count,ANY_ADVERSE_EVENT_EP_daysToInddate_min,ANY_ADVERSE_EVENT_EP_indicator,ANY_EEG_count,ANY_EEG_daysToInddate_min,ANY_EEG_indicator,CARBAMAZEPINE_daysToInddate_min,...,stdprov_Therapy (Physical)_count,stdprov_Thoracic Surgery_count,stdprov_Transplant Surgery_count,stdprov_Transportation_count,stdprov_Traumatic Surgery_count,stdprov_Treatment Center_count,stdprov_Urgent Care Facility_count,stdprov_Urology_count,stdprov_Vision Center_count,treatment_arm
carbamazepine,0.070428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
valproate,0.104927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oxcarbazepine,0.143192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lamotrigine,0.106131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
levetiracetam,0.165251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
topiramate,0.068354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gabapentin,0.040755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
phenytoin,0.166232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


carbamazepine


,ADMISSION_EP_count,ADMISSION_EP_daysToInddate_min,ADMISSION_EP_indicator,ANY_ADVERSE_EVENT_EP_count,ANY_ADVERSE_EVENT_EP_daysToInddate_min,ANY_ADVERSE_EVENT_EP_indicator,ANY_EEG_count,ANY_EEG_daysToInddate_min,ANY_EEG_indicator,CARBAMAZEPINE_daysToInddate_min,...,stdprov_Therapy (Physical)_count,stdprov_Thoracic Surgery_count,stdprov_Transplant Surgery_count,stdprov_Transportation_count,stdprov_Traumatic Surgery_count,stdprov_Treatment Center_count,stdprov_Urgent Care Facility_count,stdprov_Urology_count,stdprov_Vision Center_count,treatment_arm
id,,,,,,,,,,,,,,,,,,,,,
EGVD011444_2013-07-24,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD015050_2012-05-24,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD017943_2005-09-05,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD020081_2006-06-15,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
EGVD020784_2009-06-16,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EGVD956438_2016-04-22,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,1,0,NaN,NaN
EGVD959884_2017-05-15,0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,1,NaN,NaN
EGVD970807_2008-02-25,1,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN


In [7]:
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
#new_clf.save_model('../models/prediction/est_carba_p.xgb')

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', gamma=0.4, gpu_id=-1,
              grow_policy='depthwise', importance_type='gain',
              interaction_constraints='', learning_rate=0.05, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=4, max_leaves=0,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, ...)

In [8]:
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

In [10]:
print(rep)

auc1: 0.57 (upper: 0.65, lower: 0.48) 
 auc2: 0.65 (upper: 0.73, lower: 0.56) 
 mcnemar_test: p-value 1.000 
 delong_test: p-value 0.127 
 


# Outcome primary

In [11]:
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_epilepsy_19.01.23.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_primary")
drug_name="carbamazepine"
clf = load_prediction_model(drug_name)
X = data_preprocess(X_train,impute_drug=drug_name)
Xs = data_preprocess(X_test,impute_drug=drug_name)
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
#new_clf.save_model('../models/prediction/est_carba_new_out.xgb')
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = imputed_values[col]
c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[col] = imputed_values[col]
c:\users\anastassia kolde\documents\github\unstableepilepsy\epilepsy_prediction\epilepsy_prediction\utils.py:43: PerformanceWarning: DataFrame is highly fragm

In [12]:
print(rep)

auc1: 0.54 (upper: 0.68, lower: 0.39) 
 auc2: 0.55 (upper: 0.68, lower: 0.41) 
 mcnemar_test: p-value 1.000 
 delong_test: p-value 0.890 
 


# Using the regressor imputation

In [15]:
#col_imputer = load_imputation_model("col_carbamazepine")
regress_imputer = load_imputation_model("regress_imputer_carbamazepine.pkl")

C:\ProgramData\AnacondaEnvs\epilepsy\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LassoCV from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\AnacondaEnvs\epilepsy\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\AnacondaEnvs\epilepsy\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.1.2 when using version 1.2.0. This might lead to breaking code or inval

In [16]:
X = data_preprocess(X_train,impute_drug=drug_name,replace_mean=False)
Xs = data_preprocess(X_test,impute_drug=drug_name,replace_mean=False)
print('Do we have nan value !:',any(X.isnull()))
X = regress_imputer.transform(X)
Xs = regress_imputer.transform(Xs)
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
#new_clf.save_model('../models/prediction/est_carba_new_impute.xgb')
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

Do we have nan value !: True


ValueError: Input X contains NaN.
LassoCV does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
print(rep)